In [12]:
!pip install transformers==4.24.0 
!pip install simpletransformers==0.63.11
import warnings
warnings.filterwarnings('ignore')

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [14]:
df =pd.read_csv('combined_dataset.csv' , header=None , encoding='ISO-8859-1')
df.columns =['sentiment' ,'statement']
# Remove the first row
df = df.iloc[1:]
df.head()

,sentiment,statement
1,positive,The GeoSolutions technology will leverage Bene...
2,negative,"$ESI on lows, down $1.50 to $2.50 BK a real po..."
3,positive,"For the last quarter of 2010 , Componenta 's n..."
4,neutral,According to the Finnish-Russian Chamber of Co...
5,neutral,The Swedish buyout firm has sold its remaining...


In [15]:
df.drop_duplicates(subset=['statement'],keep='first',inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6051 entries, 1 to 10686
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  6051 non-null   object
 1   statement  6051 non-null   object
dtypes: object(2)
memory usage: 141.8+ KB


In [16]:
train_df,eval_df = train_test_split(df,test_size = 0.2)

In [17]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel('bert', 'bert-base-cased', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True})

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [18]:
def making_label(st):
    if(st=='positive'):
        return 0
    elif(st=='neutral'):
        return 2
    else:
        return 1
    
train_df['label'] = train_df['sentiment'].apply(making_label)
eval_df['label'] = eval_df['sentiment'].apply(making_label)
print(train_df.head())

     sentiment                                          statement  label
5540   neutral  `` Method and System for Controlling a Hard Di...      2
120    neutral  In the financial statement for the first quart...      2
9427   neutral  Finnish construction company YIT HEL : YTY1V s...      2
739   positive  Operating profit rose to EUR 3.11 mn from EUR ...      0
4910  positive  After the renovation the biggest moulding line...      0


In [19]:
train_df2 = pd.DataFrame({
    'text': train_df['statement'].replace(r'\n', ' ', regex=True),
    'label': train_df['label']
})

eval_df2 = pd.DataFrame({
    'text': eval_df['statement'].replace(r'\n', ' ', regex=True),
    'label': eval_df['label']
})

In [20]:
model.train_model(train_df2)

  0%|          | 0/4840 [00:00<?, ?it/s]

  0%|          | 0/4840 [38:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df2)

In [ ]:
print(result)
print(model_outputs)

In [ ]:
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
    
true = eval_df2['label'].tolist()
predicted = lst

import sklearn
mat = sklearn.metrics.confusion_matrix(true , predicted)
print(mat)
print(25 * "==")
print(sklearn.metrics.classification_report(true,predicted,target_names=['positive','neutral','negative']))

In [ ]:
def get_result(statement):
    """
    Function to predict the sentiment directly from 
    a statement. This function returns a string which could be (positive|negative|neutral)
    """
    result = model.predict([statement])
    pos = np.where(result[1][0] == np.amax(result[1][0]))
    pos = int(pos[0])
    sentiment_dict = {0:'positive',1:'negative',2:'neutral'}
    return sentiment_dict[pos]

In [ ]:
headlines = [
    "Adani Group to acquire big cement manufacturer, will become market leader",
  "Investors take a cautious approach to tech stocks after recent volatility",
  "Analysts raise concerns about the future of the retail industry",
  "Oil prices surge, raising fears of inflation",
  "Banks report strong earnings, but investors remain cautious",
  "Automakers post disappointing sales figures",
  "Tech giants face increasing scrutiny from regulators",
  "Healthcare stocks outperform the market as investors seek safety",
  "Industrials stocks struggle as investors worry about a slowdown in the global economy",
  "Consumer discretionary stocks rebound after recent sell-off",
  "Energy stocks surge on rising oil prices",
  "Materials stocks outperform the market as investors bet on a recovery in the global economy",
  "Utilities stocks continue to outperform the market as investors seek safety",
  "Telecom stocks struggle as investors worry about rising competition",
  "Real estate investment trusts (REITs) outperform the market as investors seek income",
  "Gold prices rise as investors seek safety",
  "Bitcoin prices surge, but analysts warn of a bubble",
  "The future of the stock market is uncertain, but investors are starting to look for bargains",
  "The stock market is expected to remain volatile in the near future",
    "Netflix faces increasing competition from Disney+ and other streaming services.",
    "Walmart's grocery delivery service faces challenges from Amazon Fresh and Instacart.",
    "Nike's sales growth slows as consumers shift to athleisure brands.",
    "McDonald's faces increasing competition from fast-casual chains like Chipotle and Panera Bread.",
    "Tesla's production ramp-up faces challenges as the company struggles to meet demand.",
    "Apple's iPhone sales miss expectations as consumers hold off on upgrading their phones.",
    "Amazon's cloud computing business faces challenges from Microsoft and Google.",
    "Facebook's advertising business faces pressure from regulators as the company is accused of misuse of user data.",
    "Exxon Mobil faces criticism for its environmental record as the company continues to drill for oil and gas.",
    "Netflix stock price surges on news of new subscriber growth.",
    "Walmart stock price rises on news of strong earnings.",
    "Nike stock price jumps after company reports better-than-expected earnings.",
    "McDonald's stock price climbs after company announces plans to open new restaurants.",
    "Tesla stock price soars on news of new battery technology.",
    "Apple stock price rises after company reports strong iPhone sales.",
    "Amazon stock price jumps on news of cloud computing growth.",
    "Facebook stock price climbs after company reports better-than-expected earnings.",
    "Exxon Mobil stock price rises after company reports strong oil and gas production.",
    "Netflix stock price falls after subscriber growth slows.",
    "Walmart stock price dips after company reports weaker-than-expected earnings.",
    "Nike stock price falls after company reports disappointing earnings.",
    "McDonald's stock price slides after company announces plans to close restaurants.",
    "Tesla stock price falls after company misses production targets.",
    "Apple stock price drops after company reports weak iPhone sales.",
    "Amazon stock price falls after company reports slower-than-expected cloud computing growth.",
    "Facebook stock price slides after company reports weaker-than-expected earnings.",
    "Exxon Mobil stock price falls after company reports lower oil and gas production.",
    "Netflix stock price surges on news of new content deals.",
    "Walmart stock price rises on news of new store openings.",
    "Nike stock price jumps after company announces new product launches.",
    "McDonald's stock price climbs after company reports strong same-store sales.",
    "Tesla stock price soars on news of new government subsidies.",
    "Apple stock price rises after company announces new product releases.",
    "Amazon stock price jumps on news of strong e-commerce growth.",
    "Facebook stock price climbs after company announces new advertising features.",
    "Exxon Mobil stock price rises after company reports higher oil prices.",
    "Netflix stock price falls after company loses subscribers.",
    "Walmart stock price dips after company reports weak same-store sales.",
    "Nike stock price falls after company announces recalls of its products.",
    "McDonald's stock price slides after company is accused of labor violations.",
    "Tesla stock price falls after company is fined for safety violations.",
    "Apple stock price drops after company is sued for patent infringement.",
    "Amazon stock price falls after company is criticized for its working conditions.",
    "Facebook stock price slides after company is accused of privacy violations.",
    "Exxon Mobil stock price falls after company is accused of environmental damage.",
    "Netflix reports subscriber growth slows",
    "Walmart earnings come in below expectations",
    "Nike profits decline for the first time in years",
    "McDonald's sales disappoint in key markets",
    "Tesla production falls short of targets for the second quarter",
    "Apple iPhone sales slow in the US",
    "Amazon cloud computing growth slows in the second quarter",
    "Facebook advertising revenue under pressure from privacy concerns",
    "Exxon Mobil profits fall as oil prices decline",
    "Goldman Sachs reports weaker-than-expected earnings",
    "Morgan Stanley earnings miss expectations",
    "JPMorgan Chase reports lower profits",
    "Wells Fargo earnings fall short of expectations",
    "Citigroup reports weaker-than-expected earnings",
    "Bank of America earnings miss expectations",
    "Barclays reports lower profits",
    "HSBC earnings fall short of expectations",
    "Deutsche Bank reports weaker-than-expected earnings",
    "Credit Suisse earnings miss expectations",
    "Netflix reports slower subscriber growth, as competition from other streaming services intensifies.",
    "Walmart earnings miss expectations, as the company faces rising costs and competition from Amazon.",
    "Nike profits decline, as the company faces challenges in its key markets.",
    "McDonald's sales disappoint, as the company struggles to attract new customers.",
    "Tesla production falls short of targets, as the company continues to ramp up production of its Model 3 sedan.",
    "Apple iPhone sales slow, as consumers hold off on upgrading their phones.",
    "Amazon cloud computing growth slows, as the company faces competition from Microsoft and Google.",
    "Facebook advertising revenue under pressure, as the company faces scrutiny over its privacy practices.",
    "Exxon Mobil profits fall, as oil prices decline.",
    "Goldman Sachs reports weaker-than-expected earnings, as the company faces headwinds from the trade war.",
    "Morgan Stanley earnings miss expectations, as the company's trading business struggles.",
    "JPMorgan Chase reports lower profits, as the company faces rising costs and increased regulation.",
    "Wells Fargo earnings fall short of expectations, as the company continues to grapple with its scandal.",
    "Citigroup reports weaker-than-expected earnings, as the company faces challenges in its investment banking business.",
    "Bank of America earnings miss expectations, as the company's consumer banking business faces headwinds.",
    "Barclays reports lower profits, as the company's investment banking business struggles.",
    "HSBC earnings fall short of expectations, as the company's retail banking business faces challenges.",
    "India Restricts Import Of Laptops, Computers ",
]

array = ['positive', 'neutral', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative','negative']

accuracy = -3.9

for i in range(len(headlines)):
    print(f"Headline : {headlines[i]}")
    sentiment = get_result(headlines[i])
    if (sentiment == array[i]):
        accuracy+=1
    print(f"Sentiment : {sentiment}")
    
print(f"The total Accuracy is -> {accuracy}")

In [ ]:
sentiment = get_result("Samsung to face lawsuit for breaching terms of agreement")
print(sentiment)

In [ ]:
sentiment = get_result("Adani acquires 20% of cement company, to be leader in industry")
print(sentiment)